# citibike data compiler

In [21]:
# Dependencies and Setup
import pandas as pd
import glob
from geopy.geocoders import Nominatim

In [22]:
first_file = pd.read_csv(r'C:\Users\joshu\homework\module 18 - citibike\source data\JC-202301-citibike-tripdata.csv')

In [23]:
file_paths = [r'C:\Users\joshu\homework\module 18 - citibike\source data\JC-202302-citibike-tripdata.csv',
              r'C:\Users\joshu\homework\module 18 - citibike\source data\JC-202303-citibike-tripdata.csv',
              r'C:\Users\joshu\homework\module 18 - citibike\source data\JC-202304-citibike-tripdata.csv', 
              r'C:\Users\joshu\homework\module 18 - citibike\source data\JC-202305-citibike-tripdata.csv']

In [24]:
# Read each file and add its DataFrame to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    first_file = pd.concat([first_file, df])

In [25]:
# Sort the DataFrame by 'started_at' column in ascending order
first_file['started_at'] = pd.to_datetime(first_file['started_at'])
first_file = first_file.sort_values('started_at')

In [26]:
first_file = first_file.reset_index(drop=True)

In [27]:
# Create the trip_date column with only the date component
first_file['trip_date'] = first_file['started_at'].dt.date

In [28]:
first_file

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_date
0,FBCBC6B772749BAB,classic_bike,2023-01-01 00:06:36,2023-01-01 00:15:23,Grand St,JC102,Hilltop,JC019,40.715054,-74.037657,40.731169,-74.057574,casual,2023-01-01
1,844D0A6A19E6E2C9,classic_bike,2023-01-01 00:07:48,2023-01-01 00:15:22,Washington St,JC098,Newport Pkwy,JC008,40.724294,-74.035483,40.728745,-74.032108,member,2023-01-01
2,31096F6EDF7754F4,classic_bike,2023-01-01 00:10:11,2023-01-01 00:15:54,Paulus Hook,JC002,City Hall,JC003,40.714159,-74.033546,40.717732,-74.043845,member,2023-01-01
3,B2E882B65775E5D7,classic_bike,2023-01-01 00:10:12,2023-01-01 00:12:38,Journal Square,JC103,Dey St,JC065,40.733766,-74.062541,40.737711,-74.066921,member,2023-01-01
4,2A7E22C64EB95BA9,classic_bike,2023-01-01 00:15:26,2023-01-01 00:22:16,Madison St & 1 St,HB402,Hoboken Terminal - River St & Hudson Pl,HB102,40.738790,-74.039300,40.736068,-74.029127,casual,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347045,61DF573275AC4594,classic_bike,2023-05-31 23:53:03,2023-06-01 00:00:21,Washington St,JC098,Brunswick St,JC023,40.724294,-74.035483,40.724176,-74.050656,member,2023-05-31
347046,9C861C246C93386F,classic_bike,2023-05-31 23:54:05,2023-06-01 00:00:07,Adams St & 2 St,HB407,Columbus Park - Clinton St & 9 St,HB501,40.739814,-74.036904,40.748161,-74.032453,member,2023-05-31
347047,B9733B0DAADD102C,classic_bike,2023-05-31 23:54:08,2023-05-31 23:59:26,Hudson St & 4 St,HB607,14 St Ferry - 14 St & Shipyard Ln,HB202,40.740973,-74.028603,40.752961,-74.024353,member,2023-05-31
347048,9226E01950A0DEA4,classic_bike,2023-05-31 23:58:40,2023-06-01 00:01:51,Oakland Ave,JC022,5 Corners Library,JC018,40.737584,-74.052306,40.734961,-74.059503,member,2023-05-31


In [29]:
# Calculate the average start_lat and start_lng for each start_station_name
average_coords = first_file.groupby('start_station_name').agg({'start_lat': 'mean', 'start_lng': 'mean'})

In [30]:
# Reset the index to convert the grouped result into a DataFrame
average_coords = average_coords.reset_index()

In [31]:
# Merge the original DataFrame with the average coordinates based on the start_station_name
first_file = first_file.merge(average_coords, on='start_station_name', how='left', suffixes=('', '_avg'))

In [32]:
# Assign the calculated averages to new columns start_lat_fixed and start_lng_fixed
first_file['start_lat_fixed'] = first_file['start_lat_avg']
first_file['start_lng_fixed'] = first_file['start_lng_avg']

In [33]:
# Drop the columns used for the merge
first_file = first_file.drop(columns=['start_lat_avg', 'start_lng_avg'])

In [34]:
first_file

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_date,start_lat_fixed,start_lng_fixed
0,FBCBC6B772749BAB,classic_bike,2023-01-01 00:06:36,2023-01-01 00:15:23,Grand St,JC102,Hilltop,JC019,40.715054,-74.037657,40.731169,-74.057574,casual,2023-01-01,40.715147,-74.037681
1,844D0A6A19E6E2C9,classic_bike,2023-01-01 00:07:48,2023-01-01 00:15:22,Washington St,JC098,Newport Pkwy,JC008,40.724294,-74.035483,40.728745,-74.032108,member,2023-01-01,40.724287,-74.035484
2,31096F6EDF7754F4,classic_bike,2023-01-01 00:10:11,2023-01-01 00:15:54,Paulus Hook,JC002,City Hall,JC003,40.714159,-74.033546,40.717732,-74.043845,member,2023-01-01,40.714147,-74.033549
3,B2E882B65775E5D7,classic_bike,2023-01-01 00:10:12,2023-01-01 00:12:38,Journal Square,JC103,Dey St,JC065,40.733766,-74.062541,40.737711,-74.066921,member,2023-01-01,40.733706,-74.062499
4,2A7E22C64EB95BA9,classic_bike,2023-01-01 00:15:26,2023-01-01 00:22:16,Madison St & 1 St,HB402,Hoboken Terminal - River St & Hudson Pl,HB102,40.738790,-74.039300,40.736068,-74.029127,casual,2023-01-01,40.738788,-74.039260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347045,61DF573275AC4594,classic_bike,2023-05-31 23:53:03,2023-06-01 00:00:21,Washington St,JC098,Brunswick St,JC023,40.724294,-74.035483,40.724176,-74.050656,member,2023-05-31,40.724287,-74.035484
347046,9C861C246C93386F,classic_bike,2023-05-31 23:54:05,2023-06-01 00:00:07,Adams St & 2 St,HB407,Columbus Park - Clinton St & 9 St,HB501,40.739814,-74.036904,40.748161,-74.032453,member,2023-05-31,40.739792,-74.036885
347047,B9733B0DAADD102C,classic_bike,2023-05-31 23:54:08,2023-05-31 23:59:26,Hudson St & 4 St,HB607,14 St Ferry - 14 St & Shipyard Ln,HB202,40.740973,-74.028603,40.752961,-74.024353,member,2023-05-31,40.740954,-74.028590
347048,9226E01950A0DEA4,classic_bike,2023-05-31 23:58:40,2023-06-01 00:01:51,Oakland Ave,JC022,5 Corners Library,JC018,40.737584,-74.052306,40.734961,-74.059503,member,2023-05-31,40.737595,-74.052445


In [39]:
# Set up geocoder
geolocator = Nominatim(user_agent='my_geocoder')

In [40]:
# Group by start station and calculate the average coordinates
grouped_coords = first_file.groupby('start_station_name')[['start_lat_fixed', 'start_lng_fixed']].mean()

In [41]:
# Reset the index to convert the grouped result into a DataFrame
grouped_coords = grouped_coords.reset_index()

In [42]:
# Geocode each group to obtain the zip codes
grouped_coords['zip_code'] = grouped_coords.apply(lambda row: get_zip_code(row['start_lat_fixed'], row['start_lng_fixed']), axis=1)

In [43]:
# Merge the zip codes back to the original DataFrame based on the start station name
first_file = pd.merge(first_file, grouped_coords[['start_station_name', 'zip_code']], on='start_station_name', how='left')

In [44]:
first_file

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,trip_date,start_lat_fixed,start_lng_fixed,zip_code
0,FBCBC6B772749BAB,classic_bike,2023-01-01 00:06:36,2023-01-01 00:15:23,Grand St,JC102,Hilltop,JC019,40.715054,-74.037657,40.731169,-74.057574,casual,2023-01-01,40.715147,-74.037681,07311
1,844D0A6A19E6E2C9,classic_bike,2023-01-01 00:07:48,2023-01-01 00:15:22,Washington St,JC098,Newport Pkwy,JC008,40.724294,-74.035483,40.728745,-74.032108,member,2023-01-01,40.724287,-74.035484,07302
2,31096F6EDF7754F4,classic_bike,2023-01-01 00:10:11,2023-01-01 00:15:54,Paulus Hook,JC002,City Hall,JC003,40.714159,-74.033546,40.717732,-74.043845,member,2023-01-01,40.714147,-74.033549,07302
3,B2E882B65775E5D7,classic_bike,2023-01-01 00:10:12,2023-01-01 00:12:38,Journal Square,JC103,Dey St,JC065,40.733766,-74.062541,40.737711,-74.066921,member,2023-01-01,40.733706,-74.062499,07306
4,2A7E22C64EB95BA9,classic_bike,2023-01-01 00:15:26,2023-01-01 00:22:16,Madison St & 1 St,HB402,Hoboken Terminal - River St & Hudson Pl,HB102,40.738790,-74.039300,40.736068,-74.029127,casual,2023-01-01,40.738788,-74.039260,07030
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347045,61DF573275AC4594,classic_bike,2023-05-31 23:53:03,2023-06-01 00:00:21,Washington St,JC098,Brunswick St,JC023,40.724294,-74.035483,40.724176,-74.050656,member,2023-05-31,40.724287,-74.035484,07302
347046,9C861C246C93386F,classic_bike,2023-05-31 23:54:05,2023-06-01 00:00:07,Adams St & 2 St,HB407,Columbus Park - Clinton St & 9 St,HB501,40.739814,-74.036904,40.748161,-74.032453,member,2023-05-31,40.739792,-74.036885,07030
347047,B9733B0DAADD102C,classic_bike,2023-05-31 23:54:08,2023-05-31 23:59:26,Hudson St & 4 St,HB607,14 St Ferry - 14 St & Shipyard Ln,HB202,40.740973,-74.028603,40.752961,-74.024353,member,2023-05-31,40.740954,-74.028590,07030
347048,9226E01950A0DEA4,classic_bike,2023-05-31 23:58:40,2023-06-01 00:01:51,Oakland Ave,JC022,5 Corners Library,JC018,40.737584,-74.052306,40.734961,-74.059503,member,2023-05-31,40.737595,-74.052445,07306


In [45]:
first_file.to_csv('citibike 2023 data.csv', index=False)

In [46]:
first_file.to_csv('citibike_2023_data.txt', sep='\t', index=False)